In [1]:
import numpy as np

from ase import Atoms, units
from ase.calculators.lj import LennardJones
from ase.optimize import BFGS
from ase.constraints import FixedPlane
from ase.visualize import view

from math import sqrt, exp

import matplotlib.pyplot as plt

In [2]:
# trimer along the x-axis
d = 1.0
trimer = Atoms('3Ar',[(0.0,0.0,0.0),(d,0.0,0.0),(-d,0.0,0.0)])

# getting the positions
pos = trimer.get_positions()

# modifying positions
pos += 0.1
        
# setting positions
trimer.set_positions(pos)

# getting all distances
alldist = trimer.get_all_distances()

# viewing the trimer
view(trimer)

## 3 Initial cluster setup

In [84]:
def R(θ):
    """
    2D Rotation matrix for counterclockwise
    rotaion by θ around origin 
    """
    return np.array([[np.cos(θ), -np.sin(θ)]
                    ,[np.sin(θ),  np.cos(θ)]])

In [87]:
a = 1.1
positions = np.zeros((7,3))
positions[1:,0] = a

rotations = np.array([np.linalg.matrix_power(R(np.pi/3),n) for n in range(6)])

positions[1:,:2] = np.einsum('ijk,ik->ij', rotations, positions[1:,:2])

In [88]:
cluster_2d = Atoms('7Ar',positions)
view(cluster_2d)

In [89]:
positions = np.zeros((7,3))
positions[[1,2],2] = a, -a
positions[3:,0] = a

rotations = np.array([np.linalg.matrix_power(R(np.pi/2),n) for n in range(4)])

positions[3:,:2] = np.einsum('ijk,ik->ij', rotations, positions[3:,:2])

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.10000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -1.10000000e+00],
       [ 1.10000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 6.73555740e-17,  1.10000000e+00,  0.00000000e+00],
       [-1.10000000e+00,  1.34711148e-16,  0.00000000e+00],
       [-2.02066722e-16, -1.10000000e+00,  0.00000000e+00]])

## Setting a calculator, computing the energy, and minimization

In [ ]:
# setting the calculator
ljcalc = LennardJones()
trimer.calc = ljcalc

# computing the energy
epot = trimer.get_potential_energy()

# constrain to xy-plane each atom index in trimer
trimer.set_constraint(FixedPlane(a=np.arange(len(trimer)), direction=[0,0,1]))

# setting and running a minimizer
trimer_mini = BFGS(trimer)
trimer_mini.run(fmax=0.05)

# Boltzman constant in units eV/K
units.kB

# Random numbers

In [ ]:
from numpy.random import default_rng
# a random number generator
rng = default_rng(19884)

# a sample from the gaussian distribution N(0, 1)
x_g = rng.standard_normal()

# a sample from the uniform distribution on [0,1]
x_u = rng.uniform()